The idea of this initial processing is to extract and generate places that our agent will then iteritevly check if they are real and meaningful/of interest.

In [5]:
#array where we'll store this candidate places
candidate_places = []

In [4]:
from qdrant_haystack.document_stores import QdrantDocumentStore

In [6]:
import pickle
with open('articles_db.pkl', 'rb') as f:
    document_store = pickle.load(f)

We'll extract all named entities with LOC tag.

In [7]:
docs_gen = document_store.get_all_documents_generator()

In [8]:
for doc in docs_gen:
    groups = doc.meta["entity_groups"]
    words = doc.meta["entity_words"]
    for g, w in zip(groups,words):
        if g == "LOC" and w not in candidate_places: #check if LOC and that we are not repeating a word
            candidate_places.append(w)

In [9]:
candidate_places[:10] #we got some places.

['Trondheim',
 'Norway',
 'Trøndelag',
 'Bakke',
 'Left Bank ”',
 'Gamle Bybro',
 'Old Town Bridge',
 'Kjøpmannsgata',
 'Brattørgata',
 'Nedre Elvehavn']

In [11]:
with open('initial_places.pkl','wb') as file:
    pickle.dump(candidate_places,file)

The next approach we'll take is to find extra places by searching for each of these in duckduckgo maps. We should get places in each place we are looking for...

In [39]:
from duckduckgo_search import DDGS

with DDGS() as ddgs:
    for r in ddgs.maps("school", place="Uganda"):
        print(r)

{'title': 'Victoria Cafe', 'address': 'Victoria Towers, Plot 1-13, Jinja Road, Kampala, UG ', 'country_code': None, 'latitude': 0.313616225942448, 'longitude': 32.5891662575305, 'url': 'http://www.vu.ac.ug', 'desc': None, 'phone': '+256702711473', 'image': None, 'source': 'https://www.tripadvisor.com/Restaurant_Review-g293841-d13388814-Reviews-Victoria_Cafe-Kampala_Central_Region.html?m=63959', 'links': None, 'hours': {}}
{'title': 'Makerere University', 'address': 'Makerere University, Access, Kampala, Uganda', 'country_code': 'UG', 'latitude': 0.334887981960193, 'longitude': 32.5684010982513, 'url': 'https://www.mak.ac.ug', 'desc': 'We Build for the Future', 'phone': '+256414542803', 'image': 'https://www.mak.ac.ug/sites/default/files/Mak-Logo-Mak-100-Light.png', 'source': 'https://maps.apple.com/place?q=Makerere+University&auid=190741387168377011&address=Makerere+University,+Access,+Kampala,+Uganda&ll=0.3348879819601935,32.56840109825134', 'links': '', 'hours': ''}
{'title': 'Kampal

For more information check: https://httpstatuses.com/403


{'title': 'Candlelight High School Nakasajja', 'address': 'Lake Victoria, Buvuma Island, Uganda', 'country_code': 'UG', 'latitude': -0.0709086, 'longitude': 33.0432724, 'url': 'https://www.facebook.com/candlelighthighschool?ref=py_c', 'desc': None, 'phone': '', 'image': None, 'source': 'https://maps.apple.com/place?q=Candlelight+High+School+Nakasajja&auid=7841541463945567544&address=Lake+Victoria,+Buvuma+Island,+Uganda&ll=-0.0709086,33.0432724', 'links': None, 'hours': ''}


For more information check: https://httpstatuses.com/403
For more information check: https://httpstatuses.com/403


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 from duckduckgo_search import DDGS                                                           │
│   2                                                                                              │
│   3 with DDGS() as ddgs:                                                                         │
│ ❱ 4 │   for r in ddgs.maps("school", place="Uganda"):                                            │
│   5 │   │   print(r)                                                                             │
│   6                                                                                              │
│                                                                                                  │
│ /Users/josepsmachine/miniforge3/envs/haystacky/lib/python3.8/site-packages/duckduckgo_search/duc │
│ kduckgo_search.py:816 in maps                                                                    │
│                                                                                                  │
│   813 │   │   │   │   "bbox_br": f"{lat_b},{lon_r}",                                             │
│   814 │   │   │   │   "strict_bbox": "1",                                                        │
│   815 │   │   │   }                                                                              │
│ ❱ 816 │   │   │   resp = self._get_url(                                                          │
│   817 │   │   │   │   "GET", "https://duckduckgo.com/local.js", params=params                    │
│   818 │   │   │   )                                                                              │
│   819 │   │   │   if resp is None:                                                               │
│                                                                                                  │
│ /Users/josepsmachine/miniforge3/envs/haystacky/lib/python3.8/site-packages/duckduckgo_search/duc │
│ kduckgo_search.py:89 in _get_url                                                                 │
│                                                                                                  │
│    86 │   │   │   except Exception as ex:                                                        │
│    87 │   │   │   │   logger.warning(f"_get_url() {url} {type(ex).__name__} {ex}")               │
│    88 │   │   │   │   if i >= 2 or "418" in str(ex):                                             │
│ ❱  89 │   │   │   │   │   raise ex                                                               │
│    90 │   │   │   sleep(3)                                                                       │
│    91 │   │   return None                                                                        │
│    92                                                                                            │
│                                                                                                  │
│ /Users/josepsmachine/miniforge3/envs/haystacky/lib/python3.8/site-packages/duckduckgo_search/duc │
│ kduckgo_search.py:83 in _get_url                                                                 │
│                                                                                                  │
│    80 │   │   │   │   )                                                                          │
│    81 │   │   │   │   if self._is_500_in_url(str(resp.url)) or resp.status_code == 202:          │
│    82 │   │   │   │   │   raise httpx._exceptions.HTTPError("")                                  │
│ ❱  83 │   │   │   │   resp.raise_for_status()                                                    │
│    84 │   │   │   │   if resp.status_code == 200:          